<a href="https://colab.research.google.com/github/SuYouge/colab/blob/master/mnist_tf2_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 手写数字识别tf2.0测试
主要包含以下内容：
1. 导入tf2.0
2. 启用GPU
3. 导入opencv-python以及colab专用cv2_imshow
4. 神经网络例程
5. 结果图像输出


## 1.1 导入tf2.0以及启用GPU

In [0]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
tf.device('/device:GPU:0')

## 1.2 导入Opencv
也许会报错ERROR，似乎没有大影响，暂时没有用到opencv

In [0]:
# 导入opencv
# !apt-get -qq install -y libsm6 libxext6 && pip3 install -q -U opencv-python
# import cv2
# print(cv2.__version__)

## 1.3 挂载谷歌云盘

In [0]:
import os
from google.colab import drive
drive.mount('/content/drive')

In [0]:
path = "/content/drive/My Drive/mnist_test"
os.chdir(path)
os.listdir(path)

## 1.4 mnist手写数字识别例程

### 加载并预处理数据

In [0]:
# 导入数据集

def preprocess(x, y):
    x = tf.cast(x, tf.float32) / 255.0 # 将MNIST数据映射到[0，1]
    x = tf.expand_dims(x, axis=-1) # 由于卷积层维度为[None, 28, 28, 1]，故在axis=3扩展一维
    # y = tf.one_hot(y, depth=10)
    return x, y
def load_dataset(mnist):

  (x_train, y_train), (x_test, y_test) = mnist.load_data()
  print(x_train.shape, y_train.shape, x_train.min(), x_train.max())

  train_db = tf.data.Dataset.from_tensor_slices((x_train, y_train))
  train_db = train_db.shuffle(1000).map(preprocess).batch(100)

  test_db = tf.data.Dataset.from_tensor_slices((x_test, y_test))
  test_db = test_db.map(preprocess).batch(100)

  return train_db, test_db
mnist = tf.keras.datasets.mnist
(x_train,y_train),(x_test,y_test) = mnist.load_data()
x_train, y_train = preprocess(x_train,y_train)
x_test, y_test = preprocess(x_test,y_test)
train_db, test_db= load_dataset(mnist)

### 配置Tensorboard


In [0]:
%load_ext tensorboard

### 构造网络
**Sequential模型**
1. 创建Sequential模型
2. 添加所需要的神经层
3. 使用.compile方法确定模型训练结构
4. 使用.fit方法，使模型与训练

**tf.keras.layers**

[layers的类型](https://tensorflow.google.cn/api_docs/python/tf/keras/layers)

* Flatten : Flattens the input. Does not affect the batch size.

* Dense : Just your regular densely-connected NN layer.

* Dropout : Applies Dropout to the input.

In [0]:
# 1. 构造网络
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(6,kernel_size=3,strides=1),
  tf.keras.layers.MaxPooling2D(pool_size=2,strides=2),
  tf.keras.layers.ReLU(), #(可以不要这一层)激活函数类并不是主要的网络计算层，不计入网络层数
  tf.keras.layers.Conv2D(16,kernel_size=3,strides=1),
  tf.keras.layers.MaxPooling2D(pool_size=2,strides=2),
  tf.keras.layers.ReLU(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(120,activation='relu'),
  tf.keras.layers.Dense(84,activation='relu'),
  tf.keras.layers.Dense(10) #输出层，没有激活函数（激活函数为None）
])
# 或者用这种格式
# model = tf.keras.Sequential()
# model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))
# model.add(tf.keras.layers.Dense(128, activation='relu'))
# model.add(tf.keras.layers.Dropout(0.2))
# model.add(tf.keras.layers.Dense(10))

### 定义损失函数

In [0]:
# 2. 定义损失函数
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)


### 配置训练流程以及回调功能

In [0]:
# 为网络模型定义compile参数，配置模型的学习流程
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [0]:
# 显示网络结构
model.build(input_shape=[None, 28, 28, 1])
model.summary()
filepath = 'my_model.h5' # 保存模型地址
saved_model = tf.keras.callbacks.ModelCheckpoint(filepath, verbose = 2) # 回调保存模型功能
tensorboard = tf.keras.callbacks.TensorBoard(log_dir = 'log') # 回调可视化数据功能

### 开始训练

In [0]:
# 开始训练
# model.fit(x_train, y_train, epochs=5)
history = model.fit(train_db, 
            epochs = 20, 
            validation_data = test_db, 
            validation_freq = 1,
            callbacks = [saved_model, tensorboard],
            verbose = 2)
print("\n")
# 显示训练记录
history.history

### 测试集评估训练结果




In [0]:
# 对测试集进行验证
model.evaluate(test_db, verbose=2)

### 查看TensorBoard输出

In [0]:
# 打开TensorBoard查看训练记录
%tensorboard --logdir log

### softmax结果输出

In [0]:
# 利用softmax对测试结果进行输出
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

### 查看训练效果

In [0]:
# 查看测试集上的效果
import numpy as np
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
# 显示测试集前n张的预测结果
first_n = 3
# b = probability_model(x_test[:first_n]).numpy()
# print(b.shape[0])
# 输出每行最大值的索引
# a = np.argmax(b, axis=1)  

# print(x_test[0].shape)
for i in range(first_n):
  predictions = model(x_test[i : i+1]).numpy()
  # print("预测值：\n",predictions)
  sfm = tf.nn.softmax(predictions).numpy()
  print("softmax：\n",sfm)
  # print("预测结果为：\n",a[i])
  print("\033[1;31;47m预测结果为：%s\033[0m"%np.argmax(sfm)) # softmax输出结果最大的索引为估计值
  loss = loss_fn(y_test[i : i+1], predictions).numpy()
  print("loss：\n",loss)
  x_show = tf.squeeze(x_test[i]) # 因为在开头扩展了维度，所以要缩减维度来显示图像
  plt.imshow(x_show, cmap=plt.cm.binary)
  plt.show()  